In [256]:
import numpy as np
import torch
from torch import Tensor
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import MNIST
import random
from tqdm import tqdm

import matplotlib.pyplot as plt

from CNNmodel import ConvModel

In [257]:
random_seed = 0
torch.manual_seed(random_seed)
random.seed(random_seed)
np.random.seed(random_seed)

In [258]:
DATA_DIR = "./data"
n_classes = 10
num_labelled = 125
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [259]:
def one_hot(y):
	one_hot_y = torch.zeros((len(y), 10))
	one_hot_y[np.arange(len(y)), y] = 1
	return one_hot_y

In [260]:
data = MNIST(DATA_DIR, train = True, download=True)
data_X = data.data.unsqueeze(1).float().to(device)
data_X /= 255
data_Y = one_hot(data.targets).to(device)

num_data = data_Y.shape[0]

In [261]:
def supervised_samples(X: Tensor, y: Tensor, n_samples, val_ratio = 0.2):
	num_data = y.shape[0]

	ix = np.random.randint(0, num_data, n_samples)

	val = ix[:int(n_samples*val_ratio)]
	sup = ix[int(n_samples*val_ratio):]

	X_sup = X[sup]
	y_sup = y[sup]
	X_val = X[val]
	y_val = y[val]

	return X_sup, y_sup, X_val, y_val

In [262]:
class CustomDataSet(Dataset):
	def __init__(self, x: Tensor, y: Tensor) -> None:
		self.x: Tensor = x
		self.y: Tensor = y
		self.n_samples = len(y)
	
	def __getitem__(self, index):
		return self.x[index], self.y[index]
	
	def __len__(self):
		return self.n_samples

In [263]:
class Generator(nn.Module):
	def __init__(self, inp_size, out_size) -> None:
		super().__init__()


		self.NN = nn.Sequential(
			nn.Linear(inp_size, 256*7*7),
			nn.LeakyReLU(negative_slope=0.2),
		)

		self.CONV = nn.Sequential(
			nn.ConvTranspose2d(256, 128, (3, 3), (2, 2)),
			nn.BatchNorm2d(128),
			nn.LeakyReLU(0.2),
			nn.ConvTranspose2d(128, 64, (3, 3), (1, 1)),
			nn.BatchNorm2d(64),
			nn.LeakyReLU(0.2),
			
		)

		self.out = nn.Sequential(
			nn.ConvTranspose2d(64, out_size[0], (3, 3), (2, 2)),
			nn.AdaptiveAvgPool2d((out_size[1], out_size[2])),
			nn.Sigmoid()
		)

		self.optimizer = optim.Adam(self.parameters(), lr = 0.0002, betas=[0.5, 0.999])
		self.criterion = nn.BCELoss()
	
	def forward(self, X: Tensor):
		X = self.NN(X)
		X = X.view(-1, 256, 7, 7)
		X = self.CONV(X)

		X = self.out(X)

		return X

In [264]:
class Feature_Extractor(nn.Module):
	def __init__(self, inp_channel) -> None:
		super().__init__()


		self.CNN = ConvModel(inp_channel)

		self.dropout = nn.Sequential(
			nn.Dropout(0.4)
		)

	
	def forward(self, X: Tensor):
		X = self.CNN(X)
		X = self.dropout(X)
		return X

In [265]:
class Classify(nn.Module):
	def __init__(self, feature_extractor: nn.Module, num_classes) -> None:
		super().__init__()

		self.CNN = feature_extractor

		self.out = nn.Sequential(
			nn.Linear(512, num_classes),
			nn.Softmax(1)
		)

		self.optimizer = optim.Adam(self.parameters(), lr = 0.0002, betas= [0.5, 0.999])

		self.criterion = nn.BCELoss()
	
	
	def forward(self, X: Tensor):
		X = self.CNN(X)
		X = self.out(X)

		return X

In [266]:
class Discriminator(nn.Module):
	def __init__(self, feature_extractor) -> None:
		super().__init__()
		self.CNN = feature_extractor

		self.out = nn.Sequential(
			nn.Linear(512, 1),
			nn.Sigmoid()
		)
	
		self.optimizer = optim.Adam(self.parameters(), lr=0.0002, betas=[0.5, 0.999])
		self.criterion = nn.BCELoss()

	def forward(self, X: Tensor):
		X = self.CNN(X)

		X = self.out(X)

		return X

In [267]:
class SGAN:
	def __init__(self, image_size, num_classes, feature_extractor: nn.Module, latent_size = 100, lr=0.0002):

		self.latent_size = latent_size

		CNN = feature_extractor.to(device)

		self.generator = Generator(latent_size, image_size).to(device)

		self.classify = Classify(CNN, num_classes).to(device)
		self.discriminator = Discriminator(CNN).to(device)

		self.history = {}
	
	def __call__(self, X: torch.Tensor):
		return torch.argmax(self.classify(X))
	
	def save(self, PATH = "./"):
		torch.save(self.classify.state_dict(), PATH + "/classify.pt")
		torch.save(self.discriminator.state_dict(), PATH + "/discriminator.pt")
		torch.save(self.generator.state_dict(), PATH + "/generator.pt")
	
	def validation(self, X: Tensor, y: Tensor):
		self.classify.eval()

		num_data = y.shape[0]

		run_size = 10000

		current = 0
		correct = 0
		
		while current < num_data:
			correct += torch.count_nonzero(torch.argmax(self.classify(X[current: current + run_size]), 1) == torch.argmax(y[current: current + run_size], 1))
			current += run_size

		return (correct.float().item() / num_data)


	def training_step(self, model: nn.Module, optimizer: optim.Optimizer, criterion: nn.modules.loss._Loss, X: Tensor, y: Tensor):
		out: Tensor = model(X)

		loss: Tensor = criterion(out, y)

		optimizer.zero_grad()
		loss.backward()
		optimizer.step()

		return loss
	

	def fit(self, X: Tensor, y: Tensor, sup_samples, epochs = 100, batch_size = 64):

		X_sup, y_sup, X_val, y_val = supervised_samples(X, y, sup_samples)

		datasets = CustomDataSet(X, y)

		sup_datasets = CustomDataSet(X_sup, y_sup)

		dataloader = DataLoader(datasets, batch_size=batch_size, shuffle=True)

		sup_dataloader = DataLoader(sup_datasets, batch_size=batch_size//2, shuffle=True)

		for epoch in range(epochs):
			self.classify.train()
			self.discriminator.train()
			self.generator.train()

			print(f"epoch: {epoch}\nclassify: ")
			
			# for classify
			for inputs, labels in tqdm(sup_dataloader):
				sup_loss = self.training_step(self.classify, self.classify.optimizer, self.classify.criterion, inputs.to(device), labels.to(device))

			print(f'GAN:')
			# for discriminator and generator
			for inputs, _ in tqdm(dataloader):
				real_loss = self.training_step(self.discriminator, self.discriminator.optimizer, self.discriminator.criterion, inputs, torch.ones((inputs.shape[0], 1)).to(device))

				z = torch.randn((inputs.shape[0], self.latent_size)).to(device)
				gen_out = self.generator(z)
				fake_loss = self.training_step(self.discriminator, self.discriminator.optimizer, self.discriminator.criterion, gen_out, torch.zeros(inputs.shape[0], 1).to(device))
				gen_out = self.generator(z)
				gen_loss = self.training_step(self.discriminator, self.generator.optimizer, self.discriminator.criterion, gen_out, torch.ones((inputs.shape[0], 1)).to(device))

			train_acc = self.validation(X_sup, y_sup)

			val_acc = self.validation(X_val, y_val)
			
			print(f"train acc: {train_acc*100:.2f}%, val acc: {val_acc*100:.2f}%, classification_loss: {sup_loss:.2f}, discrimination_loss: {(real_loss+fake_loss)/2:.2f}, generation_loss: {gen_loss:.2f}")


In [268]:
model = SGAN([1, 28, 28], n_classes, ConvModel(1))

In [269]:
model.fit(data_X, data_Y, sup_samples=num_labelled, epochs=100, batch_size=64)

epoch: 0
classify: 


100%|██████████| 4/4 [00:00<00:00, 366.76it/s]


GAN:


100%|██████████| 938/938 [00:13<00:00, 69.04it/s]


train acc: 15.00%, val acc: 4.00%, classification_loss: 0.30, discrimination_loss: 0.02, generation_loss: 5.16
epoch: 1
classify: 


...

train acc: 100.00%, val acc: 80.00%, classification_loss: 0.07, discrimination_loss: 0.03, generation_loss: 5.39
epoch: 65
classify: 


100%|██████████| 4/4 [00:00<00:00, 327.67it/s]


GAN:


 20%|█▉        | 187/938 [00:03<00:12, 59.83it/s]


KeyboardInterrupt: 

In [270]:
model.validation(data_X, data_Y)

0.7902